# Retrievers
A retriever is an interface that returns documents given an unstructured query. It is more general than a vector store. A retriever does not need to be able to store documents, only to return (or retrieve) them. Vector stores can be used as the backbone of a retriever, but there are other types of retrievers as well.

Retrievers accept a string query as input and return a list of Document's as output.

In [6]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings

In [7]:
loaders = [
    TextLoader("data/langchain.md"),
    TextLoader("data/langchain2.md"),
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [ ]:
# Define a text splitter that will be used to create child documents from larger parent documents.
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

# Initialize a vector store named "full_documents" which will index the child chunks of the documents.
# The OllamaEmbeddings model "snowflake-arctic-embed:33m" is used to generate embeddings for these chunks.
vectorstore = Chroma(
    collection_name="full_documents", embedding_function=OllamaEmbeddings(model="snowflake-arctic-embed:33m")
)
# Set up an in-memory storage layer that will store the parent documents.
store = InMemoryStore()

# Create a retriever that uses the previously defined vector store, document store, and child splitter.
# This retriever will be able to fetch relevant parent documents based on queries and split them into child chunks as needed.
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

In [ ]:
retriever.add_documents(docs, ids=None)
list(store.yield_keys())

In [ ]:
sub_docs = vectorstore.similarity_search("LangChian")


In [ ]:
retrieved_docs = retriever.invoke("LangChian")
len(retrieved_docs[0].page_content)


15683

## Retrieving larger chunks

In [ ]:
# This text splitter is used to create the parent documents
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
# This text splitter is used to create the child documents
# It should create documents smaller than the parent
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)
# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="split_parents", embedding_function=OllamaEmbeddings(model="snowflake-arctic-embed:33m")
)
# The storage layer for the parent documents
store = InMemoryStore()

In [ ]:
# Create a retriever that uses the previously defined vector store, document store, child splitter, and parent splitter.
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)


In [ ]:
retriever.add_documents(docs)
len(list(store.yield_keys()))


29

In [ ]:
sub_docs = vectorstore.similarity_search("LangChian")

print(sub_docs[0].page_content)

```
### Streaming[​](https://python.langchain.com/docs/tutorials/summarization/#streaming "Direct link to Streaming")


In [ ]:
retrieved_docs = retriever.invoke("LangChian")

len(retrieved_docs[0].page_content)
print(retrieved_docs[0].page_content)

*   128k token OpenAI `gpt-4o`
*   200k token Anthropic `claude-3-5-sonnet-20240620`

The chain will take a list of documents, insert them all into a prompt, and pass that prompt to an LLM:

```
from langchain.chains.combine_documents import create_stuff_documents_chainfrom langchain.chains.llm import LLMChainfrom langchain_core.prompts import ChatPromptTemplate# Define promptprompt = ChatPromptTemplate.from_messages(    [("system", "Write a concise summary of the following:\\n\\n{context}")])# Instantiate chainchain = create_stuff_documents_chain(llm, prompt)# Invoke chainresult = chain.invoke({"context": docs})print(result)
```
### Streaming[​](https://python.langchain.com/docs/tutorials/summarization/#streaming "Direct link to Streaming")

Note that we can also stream the result token-by-token:

```
for token in chain.stream({"context": docs}):    print(token, end="|")
```



### Go deeper[​](https://python.langchain.com/docs/tutorials/summarization/#go-deeper "Direct link to Go dee

## Using Retrievers in LCEL

In [20]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_ollama import ChatOllama

template = """Answer the question based only on the following context:

{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOllama(model='llama3.2:1b')


def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

print(chain.invoke("What did the president say about technology?"))


You don't need to use LangGraph to build a RAG application. The code snippet you provided is a straightforward example of how to implement two steps in an RAG application, retrieve and generate text based on user input.

LangGraph provides additional features such as visualization of the control flow of your application, which can be useful for understanding how different components interact with each other. However, for simple applications like this one, using individual component invocations is sufficient.

If you want to use LangGraph, you can create a separate graph for each component and add edges between them to represent their dependencies. Alternatively, you can use the `StateGraph` class provided by LangGraph, which allows you to define a single sequence of components that form your application.

It's worth noting that LangChain is a broader language model framework that provides a lot of utility functions and tools for building and training models. If you're looking to build 